In [2]:
import yfinance as yf
import pandas as pd 
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet  import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [3]:
hj = datetime.today().strftime('%Y-%m-%d')
data_ini = '2010-07-13'
df_btc = yf.download('BTC-USD', data_ini, hj) 
df_btc.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-08-06,"$53,991.35","$57,059.92","$53,973.27","$56,034.32","$56,034.32",49300484106
2024-08-07,"$56,040.63","$57,726.88","$54,620.51","$55,027.46","$55,027.46",41637562185
2024-08-08,"$55,030.03","$62,673.77","$54,766.73","$61,710.14","$61,710.14",45298472567
2024-08-09,"$61,728.21","$61,751.86","$59,587.86","$60,880.11","$60,880.11",33425553115
2024-08-10,"$60,881.23","$61,464.51","$60,287.57","$60,945.81","$60,945.81",15745822278


In [4]:
df_btc.reset_index(inplace=True)
df_btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,$465.86,$468.17,$452.42,$457.33,$457.33,21056800
1,2014-09-18,$456.86,$456.86,$413.10,$424.44,$424.44,34483200
2,2014-09-19,$424.10,$427.83,$384.53,$394.80,$394.80,37919700
3,2014-09-20,$394.67,$423.30,$389.88,$408.90,$408.90,36863600
4,2014-09-21,$408.08,$412.43,$393.18,$398.82,$398.82,26580100
...,...,...,...,...,...,...,...
3611,2024-08-06,"$53,991.35","$57,059.92","$53,973.27","$56,034.32","$56,034.32",49300484106
3612,2024-08-07,"$56,040.63","$57,726.88","$54,620.51","$55,027.46","$55,027.46",41637562185
3613,2024-08-08,"$55,030.03","$62,673.77","$54,766.73","$61,710.14","$61,710.14",45298472567
3614,2024-08-09,"$61,728.21","$61,751.86","$59,587.86","$60,880.11","$60,880.11",33425553115


In [5]:
df = df_btc[["Date", "Adj Close"]]
df.rename(columns = {'Date' : 'ds', 'Adj Close' : 'y'}, inplace=True)

In [6]:
# Graph preço de fechamento
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.ds, y = df.y))

In [7]:
model = Prophet(seasonality_mode='additive')
model.fit(df)

16:56:33 - cmdstanpy - INFO - Chain [1] start processing
16:56:36 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
# criar df com datas futuras
df_future = model.make_future_dataframe(periods=120)
df_future.tail()

,ds
3731,2024-12-04
3732,2024-12-05
3733,2024-12-06
3734,2024-12-07
3735,2024-12-08


In [9]:
previsao = model.predict(df_future)
previsao

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2014-09-17,$211.14,"$-7,983.89","$4,881.08",$211.14,$211.14,"$-2,144.00","$-2,144.00","$-2,144.00",$33.35,$33.35,$33.35,"$-2,177.35","$-2,177.35","$-2,177.35",$0.00,$0.00,$0.00,"$-1,932.86"
1,2014-09-18,$211.50,"$-9,024.08","$4,221.55",$211.50,$211.50,"$-2,231.40","$-2,231.40","$-2,231.40",$-16.06,$-16.06,$-16.06,"$-2,215.34","$-2,215.34","$-2,215.34",$0.00,$0.00,$0.00,"$-2,019.90"
2,2014-09-19,$211.86,"$-8,473.28","$4,507.63",$211.86,$211.86,"$-2,273.27","$-2,273.27","$-2,273.27",$-22.31,$-22.31,$-22.31,"$-2,250.96","$-2,250.96","$-2,250.96",$0.00,$0.00,$0.00,"$-2,061.41"
3,2014-09-20,$212.23,"$-8,914.39","$4,031.53",$212.23,$212.23,"$-2,300.15","$-2,300.15","$-2,300.15",$-16.20,$-16.20,$-16.20,"$-2,283.95","$-2,283.95","$-2,283.95",$0.00,$0.00,$0.00,"$-2,087.92"
4,2014-09-21,$212.59,"$-9,403.85","$4,619.14",$212.59,$212.59,"$-2,322.63","$-2,322.63","$-2,322.63",$-8.64,$-8.64,$-8.64,"$-2,314.00","$-2,314.00","$-2,314.00",$0.00,$0.00,$0.00,"$-2,110.04"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3731,2024-12-04,"$68,957.93","$60,683.90","$75,080.97","$67,034.17","$70,889.77","$-1,023.50","$-1,023.50","$-1,023.50",$33.35,$33.35,$33.35,"$-1,056.85","$-1,056.85","$-1,056.85",$0.00,$0.00,$0.00,"$67,934.43"
3732,2024-12-05,"$69,023.75","$60,503.92","$74,886.61","$67,078.25","$70,988.61","$-1,084.76","$-1,084.76","$-1,084.76",$-16.06,$-16.06,$-16.06,"$-1,068.70","$-1,068.70","$-1,068.70",$0.00,$0.00,$0.00,"$67,938.99"
3733,2024-12-06,"$69,089.58","$60,592.88","$75,630.58","$67,124.51","$71,083.92","$-1,102.58","$-1,102.58","$-1,102.58",$-22.31,$-22.31,$-22.31,"$-1,080.27","$-1,080.27","$-1,080.27",$0.00,$0.00,$0.00,"$67,987.00"
3734,2024-12-07,"$69,155.40","$60,788.28","$75,110.43","$67,157.00","$71,244.91","$-1,107.44","$-1,107.44","$-1,107.44",$-16.20,$-16.20,$-16.20,"$-1,091.24","$-1,091.24","$-1,091.24",$0.00,$0.00,$0.00,"$68,047.96"


In [10]:
previsao[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3731,2024-12-04,"$67,934.43","$60,683.90","$75,080.97"
3732,2024-12-05,"$67,938.99","$60,503.92","$74,886.61"
3733,2024-12-06,"$67,987.00","$60,592.88","$75,630.58"
3734,2024-12-07,"$68,047.96","$60,788.28","$75,110.43"
3735,2024-12-08,"$68,111.39","$61,363.79","$75,476.19"


In [11]:
# Graph 
plot_plotly(model, previsao)

In [12]:
plot_components_plotly(model, previsao)